In [3]:
# Import libaries

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix

import requests

from bs4 import BeautifulSoup




### Step 1: Create a soup object from the home page

In [22]:
url = 'https://pages.git.generalassemb.ly/rldaggie/for-scraping/'
res = requests.get(url)

soup = BeautifulSoup(res.content, 'lxml')

In [58]:
rest_names = [rest.text for rest in soup.find('tbody').findAll('a')]

In [59]:
rest_html = [rest.attrs['href'] for rest in soup.find('tbody').findAll('a')]

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and path from the `href`. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}, 
    {'name': "Applebee's", 'href': 'restaurants/2.html'},
    ...
]
```

In [174]:
restaurants = []

for num, rest in enumerate(rest_names):
    main_dict = {'name':0,'href':0}
    main_dict['name'] = (rest)
    main_dict['href'] = (rest_html[num])
    restaurants.append(main_dict)


restaurants[0]

{'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}

### Step 3: Using the `href`, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [189]:
foods = []
for rest in restaurants:
    url = 'https://pages.git.generalassemb.ly/rldaggie/for-scraping/' + str(rest['href'])
    res = requests.get(url)

    soup = BeautifulSoup(res.content, 'lxml')

    temp_list = [recipe.text.split('\n') for recipe in soup.find('table', {'class': 'table'}).find('tbody').findAll('tr')]

    food_empty = []
    for row in temp_list:
        food = {}
        food['Name'] = row[1]
        food['Category'] = row[2]
        food['Calories'] = row[3]
        food['Fat'] = row[4]
        food['Carbs'] = row[4]
        food['restaurant'] = rest['name']
        
        food_empty.append(food)

    foods.append(food_empty)
foods
    

[[{'Name': 'Original Bacon Double Cheeseburger',
   'Category': 'Burgers',
   'Calories': '760',
   'Fat': '45',
   'Carbs': '45',
   'restaurant': 'A&W Restaurants'},
  {'Name': 'Coney (Chili) Dog',
   'Category': 'Entrees',
   'Calories': '340',
   'Fat': '20',
   'Carbs': '20',
   'restaurant': 'A&W Restaurants'},
  {'Name': 'Chili Fries',
   'Category': 'French Fries',
   'Calories': '370',
   'Fat': '15',
   'Carbs': '15',
   'restaurant': 'A&W Restaurants'},
  {'Name': 'Strawberry Milkshake (small)',
   'Category': 'Shakes',
   'Calories': '670',
   'Fat': '29',
   'Carbs': '29',
   'restaurant': 'A&W Restaurants'},
  {'Name': 'A&W® Root Beer Freeze (large)',
   'Category': 'Shakes',
   'Calories': '820',
   'Fat': '18',
   'Carbs': '18',
   'restaurant': 'A&W Restaurants'},
  {'Name': 'Caramel Sundae',
   'Category': 'Desserts',
   'Calories': '340',
   'Fat': '9',
   'Carbs': '9',
   'restaurant': 'A&W Restaurants'},
  {'Name': 'Strawberry Banana Smoothee',
   'Category': 'Shak

In [151]:
temp_list = [recipe.text.split('\n') for recipe in soup.find('table', {'class': 'table'}).find('tbody').findAll('tr')]
temp_list

[['', 'Original Bacon Double Cheeseburger', 'Burgers', '760', '45', '45', ''],
 ['', 'Coney (Chili) Dog', 'Entrees', '340', '20', '26', ''],
 ['', 'Chili Fries', 'French Fries', '370', '15', '49', ''],
 ['', 'Strawberry Milkshake (small)', 'Shakes', '670', '29', '90', ''],
 ['', 'A&W® Root Beer Freeze (large)', 'Shakes', '820', '18', '150', ''],
 ['', 'Caramel Sundae', 'Desserts', '340', '9', '57', ''],
 ['', 'Strawberry Banana Smoothee', 'Shakes', '420', '6', '86', ''],
 ['', 'Chocolate Fudge Blendrrr', 'Desserts', '1010', '59', '152', ''],
 ['', 'Strawberry Limeade', 'Drinks', '420', '0', '105', ''],
 ['', 'Watermelon Slushee', 'Drinks', '270', '0', '73', ''],
 ['', 'Lime Slushee', 'Drinks', '480', '0', '119', ''],
 ['', 'A&W® Root Beer', 'Drinks', '880', '0', '232', ''],
 ['', 'Cheeseburger', 'Burgers', '420', '21', '37', ''],
 ['', 'French Fries (Regular)', 'French Fries', '310', '12', '45', ''],
 ['', 'Cheese Curds (Large)', 'Sides', '1140', '80', '54', ''],
 ['', 'Chocolate Milks

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 5,131 rows

In [199]:
df = pd.DataFrame([inner for outer in foods for inner in outer])
df

,Name,Category,Calories,Fat,Carbs,restaurant
0,Original Bacon Double Cheeseburger,Burgers,760,45,45,A&W Restaurants
1,Coney (Chili) Dog,Entrees,340,20,20,A&W Restaurants
2,Chili Fries,French Fries,370,15,15,A&W Restaurants
3,Strawberry Milkshake (small),Shakes,670,29,29,A&W Restaurants
4,A&W® Root Beer Freeze (large),Shakes,820,18,18,A&W Restaurants
...,...,...,...,...,...,...
5126,Jr. Original Chocolate Frosty™,Shakes,200,5,5,Wendy's
5127,Grilled Chicken Go Wrap,Wraps,260,10,10,Wendy's
5128,Asiago Ranch Chicken Club,Sandwiches,670,32,32,Wendy's
5129,Spicy Chicken Go Wrap,Wraps,330,16,16,Wendy's


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [200]:
df.to_csv('Lab.csv')